## 数据预处理

In [1]:
import os
import shutil

def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

In [2]:

train_filenames = os.listdir('data/train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)


rmrf_mkdir('data/train2')
os.mkdir('data/train2/cat')
os.mkdir('data/train2/dog')

rmrf_mkdir('data/test2')
os.symlink('data/test/', 'data/test2/test') 

for filename in train_cat:
    os.symlink('../../train/'+filename, 'data/train2/cat/'+filename)
    #shutil.copy('data/train/'+filename, 'data/train2/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'data/train2/dog/'+filename)
    #shutil.copy('data/train/'+filename, 'data/train2/dog/'+filename)
    

### get_diff_image
输出top50和top100情况下选出的不同照片，放到data/anormal/diff目录下，猫和狗分目录存放

In [3]:
def get_diff_image():
    class_types=['dog','cat']
    for class_type in class_types:
        filenames_50 = os.listdir('data/anormal/top50/'+class_type)
        filenames_100 = os.listdir('data/anormal/top100/'+class_type)
    
        for filename50 in filenames_50:
            if filename50 not in filenames_100:
                shutil.copy('data/anormal/top50/'+class_type+'/'+filename50, 'data/anormal/diff/'+class_type+'/'+filename50)

### move_anormal_images
在trian目录中删除被选中认为是异常的图片，即：  
data/train2/cat目录中删除data/anormal/top100/cat中的图片，  
data/train2/dog目录中删除data/anormal/top50/dog目录下的图片,  
data/train2/cat目录中删除data/anormal/diff/cat中的前6张图片.

In [3]:
#从dir1中删除dir2中的文件
def move_anormal_images(dir1, dir2, threshold=0, link_flag=True):
    filenames = os.listdir(dir2)
    count = 0
    for file in filenames:
        #rmfile=dir1+'/'+file
        rmfile=os.getcwd()+'/'+ dir1+'/'+file
        if threshold > 0 and count >= threshold:
            return
        
        count += 1
        
        if os.path.exists(rmfile):
            if link_flag:
                os.unlink(rmfile)
            else:
                os.remove(rmfile)
            print("rm {}".format(rmfile))
        
        '''
        if link_flag:
            os.unlink(rmfile)
        else:
            os.remove(rmfile)
        print("rm {}".format(rmfile))
        '''
            
    

In [4]:
move_anormal_images("data/train2/dog","data/anormal/top50/dog")
move_anormal_images("data/train2/cat","data/anormal/top100/cat")
move_anormal_images("data/train2/cat","data/anormal/diff/cat", threshold=6, link_flag=False)

#move_anormal_images("data/train2/dog","data/anormal/top50/dog", link_flag=False)
#move_anormal_images("data/train2/cat","data/anormal/top100/cat", link_flag=False)
#move_anormal_images("data/train2/cat","data/anormal/diff/cat", threshold=6, link_flag=False)


rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.11266.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.3889.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.9517.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.10237.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.9188.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.6475.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.5604.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.10161.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.1895.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.10801.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.1773.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.11299.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.2614.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.1259.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.10747.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.2422.jpg
rm /home/ubuntu/dogs-and-cats/data/train2/dog/dog.

### 分训练集和验证集
data/train2/cat 目录下有12451张图片  
data/train2/dog 目录下有12479张图片  
分别取20%用作验证集数据（四舍五入），分完后，目录情况如下：  
data/validation/cat 2490张图片  
data/validation/dog 2496张图片  
data/train3/cat    9961张图片  
data/train3/dog    9983张图片  

In [5]:
#含link文件的复制
def mycopy(src, dst):
    if os.path.islink(src):
        linkto = os.readlink(src)
        os.symlink(linkto, dst)
    else:
        shutil.copy(src,dst)

#前N张分到验证集，其余放到验证集
def split_validation(soucre_dir,calss_type, rito):
    filenames = os.listdir(soucre_dir)
    total=len(filenames)
    valid_cnt=round(total*rito)
    
    valid_dir='data/validation/'+calss_type
    train_dir='data/train3/'+calss_type
    
    count=0
    for file in filenames:
        if count<valid_cnt:
            #os.symlink(soucre_dir+'/'+file, valid_dir+'/'+file)
            #shutil.copy(soucre_dir+'/'+file, valid_dir+'/'+file)  #这里应该用copy，soucre_dir中的文件已经是Link文件
            mycopy(soucre_dir+'/'+file, valid_dir+'/'+file)
        else:
            #os.symlink(soucre_dir+'/'+file, train_dir+'/'+file)
            #shutil.copy(soucre_dir+'/'+file, train_dir+'/'+file)
            mycopy(soucre_dir+'/'+file, train_dir+'/'+file)
        count += 1
    

In [6]:
rmrf_mkdir('data/validation')
os.mkdir('data/validation/cat')
os.mkdir('data/validation/dog')

rmrf_mkdir('data/train3')
os.mkdir('data/train3/cat')
os.mkdir('data/train3/dog')

split_validation('data/train2/cat','cat',rito=0.2)
split_validation('data/train2/dog','dog',rito=0.2)

In [1]:
#把文件名在dir3中的文件，从dir1复制到dir2,因为dir3中link文件
def copy_file_from1to2in3(dir1,dir2,dir3):
    filenames = os.listdir(dir3)
    
    for file in filenames:
        shutil.copy(dir1+'/'+file, dir2+'/'+file)

In [12]:
rmrf_mkdir('data/train4')
os.mkdir('data/train4/cat')
os.mkdir('data/train4/dog')

copy_file_from1to2in3('data/train', 'data/train4/cat', 'data/train3/cat')
copy_file_from1to2in3('data/train', 'data/train4/dog', 'data/train3/dog')


In [2]:
def shusu(N):
    for i in range(N-2):
        result=N//(i+2)
        if N == (i+2)*result:
            print (i+2)
            


In [53]:
shusu(19944)
shusu(4986)

2
3
6
9
18
277
554
831
1662
2493
